# 0.0 IMPORT

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display  import HTML

## 0.1 Helper Function

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [3]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2 Loading Data

In [4]:
# Whats is low_memory?
df_raw = pd.read_csv('data/vehicles.csv', low_memory=False)

In [5]:
df_raw.sample()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,county,state,lat,long
343889,7111002539,https://fresno.craigslist.org/ctd/d/fresno-201...,fresno / madera,https://fresno.craigslist.org,14900,2014.0,subaru,forester,NaN,NaN,gas,106873.0,clean,automatic,JF2SJAHC7EH486388,NaN,NaN,NaN,white,https://images.craigslist.org/00u0u_6e9pgdYiJ8...,Fresno Acura address: 7250 N Palm Avenue Fr...,NaN,ca,36.8414,-119.807


# 1.0 STEP 01 - DESCRIPTION OF DATA

In [84]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [7]:
df1.columns

# The columns already have a label that I want and easy to understand.

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'vin', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long'],
      dtype='object')

## 1.2 Data Dimensions

In [8]:
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Number of Cols: {}'.format( df1.shape[1] ) )
# Evaluate the possibilite do use this project in your computer

Number of Rows: 435849
Number of Cols: 25


## 1.3 Data Types

In [9]:
df1.dtypes
# At first, the types of the variables are corrected.

id                int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
vin              object
drive            object
size             object
type             object
paint_color      object
image_url        object
description      object
county          float64
state            object
lat             float64
long            float64
dtype: object

## 1.4. Check NA

In [10]:
df1.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1117
manufacturer     20747
model             6199
condition       186806
cylinders       166384
fuel              2991
odometer         75148
title_status      1806
transmission      2146
vin             196652
drive           122011
size            295961
type            117108
paint_color     135247
image_url           24
description         27
county          435849
state                0
lat               8235
long              8235
dtype: int64

## 1.5 Fillout NA

In [11]:
# --> year

# The first variable that has an empty value and that I will deal with is the "year".
# Looking at the other variables to be able to see if I can extract some info that helps to fill in the "nan" values, I find that the "description" column has the info year inside it.
# The problem is that there are also "nan" values in the "description" column. The positive info is that there are only 27 empty values in the "description" column.
# I only looked at the observations where "description" is empty and found that, of the 27 empty values, 3 already have the "year" column filled in and the rest have almost no value filled in the other columns.
# My solution is as follows: fill the column "year" with the information contained in the column "description" not empty and drop the entire line in the 24 observations where the column "description" is empty along with several others, as we will not have enough info.
# First of all, I will drop the rows where the columns "year" and "description" are null.
df1.dropna(subset=['year', 'description'], how='all', inplace=True)
df1['year'] = df1.apply( lambda x: x['description'][:5] if math.isnan( x['year'] ) else x['year'], axis=1 )
# After fill the nan values in "year" column, there are some values that aren't year values, such as: '92 to', '03 je', 'Auto' and 'Nice'. I will transform these values by hand.
df1['year'] = df1.apply( lambda x: 1992 if x['year']=='92 to' else x['year'], axis=1 ) # We can see easily that the year is "1992"
df1['year'] = df1.apply( lambda x: 2003 if x['year']=='03 je' else x['year'], axis=1 ) # We can see easily that the year is "2003"
df1.drop(df1.loc[df1['year']=='Auto '].index, inplace=True) # In the description columns don't have any info about the year; so, we will drop the entire row.
df1.drop(df1.loc[df1['year']=='Nice '].index, inplace=True) # In the description columns don't have any info about the year; so, we will drop the entire row.

# --> manufacturer

# Percebi que a coluna "description" e "model" possuem info que podem me ajudar a preencher a coluna "manufacturer".
# Como para tirar a info da coluna "description" vai ser mais trabalhoso em um primeiro momento, eu vou tentar fazer um "de para" da coluna "model" para preencher a coluna "manufacturer".
df1['model'] = df1['model'].str.lower() # First I will apply a lower function in the "model" column.
# In the first cicle I will drop que rows that have 'model' and 'manufacturer' with nan values at the same time.
df1.drop( index=df1[df1['manufacturer'].isna() & df1['model'].isna()].index, inplace=True )
# Now I will create a dictionary with the model and its respective manufacturer.
df11 = df1[df1['manufacturer'].notna()]
df12 = df11[df11['model'].notna()]
model_manufacturer = {}
for k, v in df12[['model', 'manufacturer']].values:
    model_manufacturer[k] = v
df1['manufacturer'] = df1.apply( lambda x: model_manufacturer[x['model']] if x['model'] in model_manufacturer.keys() else x['manufacturer'], axis=1) # reduce the "nan" values to 17848, but not resolved.

# All the "county" column have null value. I will drop this column.
df1.drop( df1['county'], axis=1, inplace=True )

In [12]:
df1.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     17815
model             6141
condition       186756
cylinders       166334
fuel              2967
odometer         75116
title_status      1782
transmission      2122
vin             196617
drive           121961
size            295934
type            117058
paint_color     135209
image_url            0
description          3
county          435790
state                0
lat               8211
long              8211
dtype: int64

In [19]:
df1[['year', 'condition', 'description']].sample(10)

,year,condition,description
379743,1972,NaN,Used ford c-4 transmission work great has ever...
115064,2009,NaN,-SUV Interior and Interior are in Very Good Co...
404478,2016,NaN,"2016 Ford F-150 XLT -- $24,482 Call Or TEXT Us..."
247121,2006,NaN,2006 Toyota Highlander Automatic transmission ...
282574,2006,excellent,"2006 Chevrolet *Silverado* *3500* *WT*41,680 m..."
376768,2019,NaN,2019 CHEVROLET IMPALA LT Offered by: Fast ...
428506,2006,NaN,Call 985-626-1948 or 504-453-7844 Cars-Trucks...
313132,2014,NaN,💥 FREE DELIVERY RIGHT TO YOUR FRONT DOOR / EAS...
293530,2019,NaN,Elite Motor Cars of MiamiAsk for: Sales Team C...
412212,2013,NaN,DKR IncAsk for: Paul or Randy☎ (817) 239-73531...


In [85]:
df1['condition'] = df1['condition'].replace( 'like new', 'new' )

In [86]:
df1['condition'].value_counts()

excellent    118442
good          93983
new           28947
fair           6971
salvage         700
Name: condition, dtype: int64

In [ ]:
df1['condition'] = df1['condition'].str.lower()
df1['description'] = df1['description'].str.lower()

In [72]:
df1['condition'] = df1.apply( lambda x: x['condition'] if pd.isna( x['description'] ) else 'excellent' if 'excellent' in x['description'].split() else 'good' if 'good' in x['description'].split() else 'fair' if 'fair' in x['description'].split() else 'new' if 'new' in x['description'].split() else 'salvage' if 'salvage' in x['description'].split() else str(x['condition']), axis=1 )

In [71]:
df1['manufacturer'] = df1.apply( lambda x: x['manufacturer'] if pd.isna( x['description'] ) else 'jeep' if 'jeep' in str(x['description']) else 'gmc' if 'gmc' in str(x['description']) else 'ram' if 'ram' in str(x['description']) else 'dodge' if 'dodge' in str(x['description']) else 'mercedes-benz' if 'mercedes-benz' in str(x['description']) else str(x['manufacturer']), axis=1 )

In [77]:
df1['condition'].value_counts()

good         151715
excellent    102535
new           94674
nan           72747
fair          11541
salvage        2578
Name: condition, dtype: int64

In [76]:
df1['condition'].isna().sum()

0

In [ ]:
df1['manufacturer'].isna().sum()